<a href="https://colab.research.google.com/github/SeongDeokKo/Replication/blob/master/Krusell_Smith_%EB%94%B0%EB%9D%BC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools PyCall PyPlot QuantEcon JLD2 Roots LinearAlgebra Plots Distributions Dierckx Optim Roots GLM StatsBase Parameters Interpolations"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2021-07-12 01:01:28 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package PyCall...
Installing Julia package PyPlot...
Installing Julia package QuantEcon...
Installing Julia package JLD2...
Installing Julia package Roots...
Installing Julia package LinearAlgebra...
Installing Julia package Plots...
Installing Julia package Distributions...
Installing Julia package Dierckx...
Installing Julia package Optim...
Installing Julia package Roots...
Installing Julia package GLM...
Installing Julia package StatsBase...
Installing Julia package Parameters...
Installing Julia package Interpolations...
Installing Julia package CUDA...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Please 

In [ ]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


In [1]:
using LinearAlgebra
using QuantEcon
using Roots
using Dierckx
using Optim
using Parameters
using StatsBase
using GLM
using Interpolations

In [2]:
const β = 0.99
const δ = 0.025
const σ = 1
const α = 0.36
const zg = 1.01
const zb = 0.99
const ug = 0.04
const ub = 0.1
const nz = 2 # the number of aggregate states
const zgrid = [zg, zb] # TFP grid
const T = 1100 # final time period
const T0 = 100 # discard all the points before this period
const ne = 2
# transition probabilities
const πgg = 7/8 # good -> good
const πgb = 1/8 # good -> bad
const πbg = 1/8 # bad -> good
const πbb = 7/8 # bad -> bad
# constant unemployment rates in each state
const ug = 0.04
const ub = 0.1
const L_bar = 1/0.9 # 이게 논문에는 안 나와있는데 코드 보니깐 이렇게 임의로 주어졌더라구요;;
const Lgrid = [1-ug, 1-ub] * L_bar # Aggregate Labor
const mu = 0.15 # Fractrion of 

0.15

In [3]:
kss=((1/β-(1-δ))/α)^(1/(α-1))

37.98925353815222

In [4]:
#=P = [P_GEGE P_GEGU P_GEBE P_GEBU ;
      P_GUGE P_GUGU P_GUBE P_GUBU ;
      P_BEGE P_BEGU P_BEBE P_BEBU ;
      P_BUGE P_BUGU P_BUBE P_BUBU ] 
=##

P = zeros(4,4); # Transition Matrix 저장할 곳 
P[2,2] = 1/3 ;
P[2,1] = 7/8 - P[2,2]
P[4,4] = 0.6 ;
P[4,3] = 7/8 - 0.6
P[2,4] = 3/28 ; 
P[2,3] = 1/8 - P[2,4]
P[4,2] = 1/28 ; 
P[4,1] = 1/8 - 1/28
RHS12 =  ug - ug * P[2,2]/πgg ; 
LHS12 = (1-ug)/πgg
P[1,2] = RHS12/LHS12 ; 
P[1,1] = 7/8 - P[1,2]
RHS14 = ub - ug*P[2,4]/πgb ; 
LHS14 = (1-ug)/πgb
P[1,4] = RHS14/LHS14 ; 
P[1,3] = 1/8 - P[1,4]
RHS32 = 0.04 - P[4,2]*8*0.1 ; 
LHS32 = 0.9/(1/8)
P[3,2] = RHS32/LHS32 ; 
P[3,1] = 1/8 - P[3,2]
RHS34 = 0.1 - P[4,4]*8/7*0.1 ; 
LHS34 = 0.9/(7/8)
P[3,4] = RHS34/LHS34 ; 
P[3,3] = 7/8 - P[3,4];


In [5]:
prob=[0.525 0.35 0.03125 0.09375  
   0.038889 0.836111 0.002083 0.122917
   0.09375 0.03125 0.291667 0.583333
   0.009115 0.115885 0.024306 0.850694];

In [6]:
prob

4×4 Matrix{Float64}:
 0.525     0.35      0.03125   0.09375
 0.038889  0.836111  0.002083  0.122917
 0.09375   0.03125   0.291667  0.583333
 0.009115  0.115885  0.024306  0.850694

In [7]:
P[1,1] = prob[4,4] ; P[1,2] = prob[4,3] ;
P[1,3] = prob[4,2] ; P[1,4] = prob[4,1] ; 
P[2,1] = prob[3,4] ; P[2,2] = prob[3,3] ; 
P[2,3] = prob[3,2] ; P[2,4] = prob[3,1] ; 
P[3,1] = prob[2,4] ; P[3,2] = prob[2,3] ;
P[3,3] = prob[2,2] ; P[3,4] = prob[2,1] ;
P[4,1] = prob[1,4] ; P[4,2] = prob[1,3] ;
P[4,3] = prob[1,2] ; P[4,4] = prob[1,1] ;

In [8]:
P

4×4 Matrix{Float64}:
 0.850694  0.024306  0.115885  0.009115
 0.583333  0.291667  0.03125   0.09375
 0.122917  0.002083  0.836111  0.038889
 0.09375   0.03125   0.35      0.525

In [9]:
using Distributions
function Generate_Shock(T::Int64,NH::Int64)
    zseq = zeros(Int64,T)
    zseq[1] = 0 # arbitrarily chosen
    
    P_GGEE = P[1,1] /(P[1,1] + P[1,2] )
    P_GBEE = P[1,3] /(P[1,3] + P[1,4] )
    P_GGUE = P[2,1] /(P[2,1] + P[2,2] )
    P_GBUE = P[2,3] /(P[2,3] + P[2,3] )

    P_BGEE = P[3,1] /(P[3,1] + P[3,2] ) 
    P_BBEE = P[3,3] /(P[3,3] + P[3,4] ) 
    P_BGUE = P[4,1] /(P[4,1] + P[4,2] ) 
    P_BBUE = P[4,3] /(P[4,4] + P[4,4] ) 
    for iter in 2:T
        if zseq[iter-1] == 1
            zseq[iter] = sample([1, 0], weights([πgg, πgb]))
        elseif zseq[iter-1] == 0
            zseq[iter] = sample([1, 0], weights([πbg, πbb]))
        end
    end

    Zt = zeros(Float64,T);
    for t in eachindex(zseq)
        if zseq[t] == 1
            Zt[t] = zgrid[1]
        else
            Zt[t] = zgrid[2]
        end 
    end

    Ind_shock = zeros(Int64,NH,T)
    for NH_idx = 1:NH
        u = rand(Uniform(0,1))
        if zseq[1] == 0
            if u < ub
                Ind_shock[NH_idx,1] = 0
            else
                Ind_shock[NH_idx,1] = 1
            end
        else zseq[1] == 1
            if u < ug
                Ind_shock[NH_idx,1] = 0
            else
                Ind_shock[NH_idx,1] = 1
            end
        end
    end
    for t = 2:T
        for NH_idx = 1:NH
            u = rand(Uniform(0,1))
            ## Good To Good 
            if zseq[t-1] == 1 && zseq[t] == 1
                if Ind_shock[NH_idx,t-1] == 1
                    if u < P_GGEE 
                        Ind_shock[NH_idx,t] = 1
                    else 
                        Ind_shock[NH_idx,t] = 0
                    end
                else 
                    if u < P_GGUE 
                        Ind_shock[NH_idx,t] = 1
                    else 
                        Ind_shock[NH_idx,t] = 0
                    end
                end
            elseif zseq[t-1] == 1 && zseq[t] == 0
            ## Good To Bad
                if Ind_shock[NH_idx,t-1] == 1
                    if u < P_GBEE
                        Ind_shock[NH_idx,t] = 1
                    else 
                        Ind_shock[NH_idx,t] = 0
                    end
                else 
                    if u < P_GBUE
                        Ind_shock[NH_idx,t] = 1
                    else 
                        Ind_shock[NH_idx,t] = 0
                    end
                end
            elseif zseq[t-1] == 0 && zseq[t] == 1
            ## Bad to Good
                if Ind_shock[NH_idx,t-1] == 1
                    if u < P_BGEE
                        Ind_shock[NH_idx,t] = 1
                    else 
                        Ind_shock[NH_idx,t] = 0
                    end
                else 
                    if u < P_BGUE 
                        Ind_shock[NH_idx,t] = 1
                    else 
                        Ind_shock[NH_idx,t] = 0
                    end
                end
            else
            ## Bad to Bad
                if Ind_shock[NH_idx,t-1] == 1
                    if u < P_BBEE
                        Ind_shock[NH_idx,t] = 1
                    else 
                        Ind_shock[NH_idx,t] = 0
                    end
                else 
                    if u < P_BBUE 
                        Ind_shock[NH_idx,t] = 1
                    else 
                        Ind_shock[NH_idx,t] = 0
                    end
                end
            end
        end
    end

    return zseq, Zt , Ind_shock

end

Generate_Shock (generic function with 1 method)

In [75]:
NH = 20_000;
zseq , Zt , Ind_shock=Generate_Shock(T,NH);

In [76]:
function grid_Cons_Grow(n::Int64, left::Real, right::Real, growth::Float64)
    h = (right-left)/((1+growth)^(n-1)-1)
    # ! calculate grid value
    return [left + h*((1+growth)^i-1) for i in 0:(n-1)]
end

grid_Cons_Grow (generic function with 1 method)

In [77]:
function U(x::Float64)
    if x < 0
        return -Inf
    end
    return log(x)
end
function U(x::Int64)
    if x < 0
        return -Inf
    end
    return log(x)
end

U (generic function with 2 methods)

In [78]:
# capital grid
Kmin = 25
Kmax = 60
nK = 8
Kgrid = range(Kmin, Kmax, length=nK) |> collect ;

In [79]:
amin=0;                #   % minimum grid-value of capitala
amax=1000;             #   % maximum grid-value of capital
na= 300;                #% number of grid points
x = range(0,0.5,length = na);# % generate a grid of ngridk points on [0,0.5] #   % interval  
y = x.^7/maximum(x).^7

agrid = amin .+ (amax - amin) * y;

In [80]:
# 초기값 
Coef= [0.0 1.0;  0 1]

2×2 Matrix{Float64}:
 0.0  1.0
 0.0  1.0

In [81]:
function Neville(x::Array{Float64}, y::Array{Float64}, x0::Float64, N::Int64)   
  
    Q = zeros(N,N)
    Q[:,1] .= y
  
    for i in 2:N 
        for j in 2:i
            Q[i,j] = ((x0 - x[i-j+1]) * Q[i,j-1] - (x0 - x[i]) * Q[i-1,j-1]) / (x[i] - x[i-j+1])
        end
    end
    return Q[N,N]
end
  
function Neville(x::Array{Float64}, y::Array{Float64}, x0::Int64, N::Int64)   
  
    Q = zeros(N,N)
    Q[:,1] .= y
    for i in 2:N 
        for j in 2:i
            Q[i,j] = ((x0 - x[i-j+1]) * Q[i,j-1] - (x0 - x[i]) * Q[i-1,j-1]) / (x[i] - x[i-j+1])
        end
    end
    return Q[N,N]
end

Neville (generic function with 2 methods)

In [82]:
function TV( V::Array{Float64,4},
    Kgrid::Array{Float64,1},
    agrid::Array{Float64,1},
    zgrid::Array{Float64,1},
    P::Array{Float64,2},
    na::Int64,
    nK::Int64,
    ne::Int64,
    nz::Int64, Coef::Array{Float64,2},U,Neville)

    Vnew = copy(V)
    gg = zeros(na,nK,nz,ne)

### Where to assign Interpolation Value
    V_GE = zeros(na);
    V_GU = zeros(na);
    V_BE = zeros(na);
    V_BU = zeros(na);
###

    for (Kidx, Kval) in enumerate(Kgrid)

        r_g = α*zg*Kval^(α-1)*(Lgrid[1])^(1-α) - δ
        w_g = (1-α) * zg * Kval^α * (Lgrid[1])^(-α)
            
        r_b = α*zb*Kval^(α-1)*(Lgrid[2])^(1-α) - δ
        w_b = (1-α) * zb * Kval^α * (Lgrid[2])^(-α)

        tau_g = mu * ug / Lgrid[1]
        tau_b = mu * ub / Lgrid[2]

        K_Prime_g = ( Coef[1,1] + Coef[1,2]*log(Kval) ) |> exp
        K_Prime_b = ( Coef[2,1] + Coef[2,2]*log(Kval) ) |> exp
        # 여기 파트에서 
              
        K_Prime_g = max(K_Prime_g, Kgrid[1])
        K_Prime_b = max(K_Prime_b, Kgrid[1])
        K_Prime_g = min(K_Prime_g, Kgrid[end])
        K_Prime_b = min(K_Prime_b, Kgrid[end])
        # Use Neville Interpolation

        V_BU = map( x ->  Neville(Kgrid, V[x,:,1,1], K_Prime_b , nK), 1:na)
        V_BE = map( x ->  Neville(Kgrid, V[x,:,1,2], K_Prime_b , nK), 1:na)
        V_GU = map( x ->  Neville(Kgrid, V[x,:,2,1], K_Prime_g , nK), 1:na)
        V_GE = map( x ->  Neville(Kgrid, V[x,:,2,2], K_Prime_g , nK), 1:na)

        # na, nK, nz, ne
        # Use Cubic Spline

        V_Next_GE = Spline1D( agrid, V_GE ,k = 3) 
        V_Next_GU = Spline1D( agrid, V_GU ,k = 3) 
        V_Next_BE = Spline1D( agrid, V_BE ,k = 3) 
        V_Next_BU = Spline1D( agrid, V_BU ,k = 3) 
    
         
        for a_idx in eachindex(agrid)

            ## Bad && Unemployed

            obj_BU(a_prime) = -(  U( mu * w_b + (1 + r_b) * agrid[a_idx] - a_prime + eps() ) + 
                β* ( P[4,1] * V_Next_GE(a_prime) + P[4,2] * V_Next_GU(a_prime) +
                P[4,3] * V_Next_BE(a_prime) + P[4,4] * V_Next_BU(a_prime))  )
            ubd_BU = min.( ( 1 + r_b )*agrid[a_idx], agrid[end] )
            res_BU = optimize(obj_BU, agrid[1], ubd_BU)       
            gg[a_idx,Kidx,1,1] = res_BU.minimizer
            Vnew[a_idx,Kidx,1,1] = (-1) * res_BU.minimum     

            ## Bad && Employed

            obj_BE(a_prime) = -(  U( w_b * L_bar*(1-tau_b) + (1 + r_b) * agrid[a_idx] - a_prime + eps() ) + 
                β * ( P[3,1] * V_Next_GE(a_prime) + P[3,2] * V_Next_GU(a_prime) +
                P[3,3] * V_Next_BE(a_prime) + P[3,4] * V_Next_BU(a_prime))    )
            ubd_BE = min.( w_b * L_bar + (1 + r_b)*agrid[a_idx], agrid[end] )
            res_BE = optimize(obj_BE, agrid[1], ubd_BE)       
            gg[a_idx,Kidx,1,2] = res_BE.minimizer
            Vnew[a_idx,Kidx,1,2] = (-1) * res_BE.minimum  


            ## Good && Unemployed

            obj_GU(a_prime) = -(  U( mu * w_g + (1 + r_g) * agrid[a_idx] - a_prime + eps() ) + 
                β* ( P[2,1] * V_Next_GE(a_prime) + P[2,2] * V_Next_GU(a_prime) +
                P[2,3] * V_Next_BE(a_prime) + P[2,4] * V_Next_BU(a_prime))  )
            ubd_GU = min.( (1 + r_g)*agrid[a_idx], agrid[end] )
            res_GU = optimize(obj_GU, agrid[1], ubd_GU)       
            gg[a_idx,Kidx,2,1] = res_GU.minimizer
            Vnew[a_idx,Kidx,2,1] = (-1) * res_GU.minimum      


            ## Good && Employed
            obj_GE(a_prime) = -(  U( w_g*L_bar*(1-tau_g) + (1 + r_g) * agrid[a_idx] - a_prime + eps() ) + 
                β* ( P[1,1] * V_Next_GE(a_prime) + P[1,2] * V_Next_GU(a_prime) +
                P[1,3] * V_Next_BE(a_prime) + P[1,4] * V_Next_BU(a_prime))   )
            ubd_GE = min.( w_g * L_bar + (1 + r_g)*agrid[a_idx], agrid[end] )
            res_GE = optimize(obj_GE, agrid[1], ubd_GE)       
            gg[a_idx,Kidx,2,2] = res_GE.minimizer
            Vnew[a_idx,Kidx,2,2] = (-1) * res_GE.minimum
        end
    end

    return Vnew, gg
end

TV (generic function with 1 method)

In [83]:
function TV_Schumaker( V::Array{Float64,4},
    Kgrid::Array{Float64,1},
    agrid::Array{Float64,1},
    zgrid::Array{Float64,1},
    P::Array{Float64,2},
    na::Int64,
    nK::Int64,
    ne::Int64,
    nz::Int64, Coef::Array{Float64,2},U,Neville)

    Vnew = copy(V);
    gg = zeros(na,nK,nz,ne)

### Where to assign Interpolation Value
    V_GE = zeros(na)
    V_GU = zeros(na)
    V_BE = zeros(na)
    V_BU = zeros(na)
###

    for (Kidx, Kval) in enumerate(Kgrid)

        r_g = α*zg*Kval^(α-1)*(Lgrid[1])^(1-α) - δ
        w_g = (1-α) * zg * Kval^α * (Lgrid[1])^(-α)
            
        r_b = α*zb*Kval^(α-1)*(Lgrid[2])^(1-α) - δ
        w_b = (1-α) * zb * Kval^α * (Lgrid[2])^(-α)

        tau_g = mu * ug / Lgrid[1]
        tau_b = mu * ub / Lgrid[2]

        K_Prime_g = ( Coef[1,1] + Coef[1,2]*log(Kval) ) |> exp
        K_Prime_b = ( Coef[2,1] + Coef[2,2]*log(Kval) ) |> exp
        # 여기 파트에서 
              
        K_Prime_g = max(K_Prime_g, Kgrid[1])
        K_Prime_b = max(K_Prime_b, Kgrid[1])
        K_Prime_g = min(K_Prime_g, Kgrid[end])
        K_Prime_b = min(K_Prime_b, Kgrid[end])


        # Use Neville Interpolation
        
        V_BU = map( x ->  Neville(Kgrid, V[x,:,1,1], K_Prime_b , nK), 1:na)
        V_BE = map( x ->  Neville(Kgrid, V[x,:,1,2], K_Prime_b , nK), 1:na)
        V_GU = map( x ->  Neville(Kgrid, V[x,:,2,1], K_Prime_g , nK), 1:na)
        V_GE = map( x ->  Neville(Kgrid, V[x,:,2,2], K_Prime_g , nK), 1:na)

        # Use Cubic Spline

        V_Next_GE = SchumakerSpline.Schumaker( agrid, V_GE ) 
        V_Next_GU = SchumakerSpline.Schumaker( agrid, V_GU ) 
        V_Next_BE = SchumakerSpline.Schumaker( agrid, V_BE ) 
        V_Next_BU = SchumakerSpline.Schumaker( agrid, V_BU )
    
         
        for a_idx in eachindex(agrid)

            ## Bad && Unemployed

            obj_BU(a_prime) = -(  U( mu * w_b + (1 + r_b) * agrid[a_idx] - a_prime + eps() ) + 
                β* ( P[4,1] * V_Next_GE(a_prime) + P[4,2] * V_Next_GU(a_prime) +
                P[4,3] * V_Next_BE(a_prime) + P[4,4] * V_Next_BU(a_prime))  )
            ubd_BU = min.( ( 1 + r_b )*agrid[a_idx], agrid[end] )
            res_BU = optimize(obj_BU, agrid[1], ubd_BU)       
            gg[a_idx,Kidx,1,1] = res_BU.minimizer
            Vnew[a_idx,Kidx,1,1] = (-1) * res_BU.minimum     

            ## Bad && Employed

            obj_BE(a_prime) = -(  U( w_b * L_bar*(1-tau_b) + (1 + r_b) * agrid[a_idx] - a_prime + eps() ) + 
                β * ( P[3,1] * V_Next_GE(a_prime) + P[3,2] * V_Next_GU(a_prime) +
                P[3,3] * V_Next_BE(a_prime) + P[3,4] * V_Next_BU(a_prime))    )
            ubd_BE = min.( w_b * L_bar + (1 + r_b)*agrid[a_idx], agrid[end] )
            res_BE = optimize(obj_BE, agrid[1], ubd_BE)       
            gg[a_idx,Kidx,1,2] = res_BE.minimizer
            Vnew[a_idx,Kidx,1,2] = (-1) * res_BE.minimum  


            ## Good && Unemployed

            obj_GU(a_prime) = -(  U( mu * w_g + (1 + r_g) * agrid[a_idx] - a_prime + eps() ) + 
                β* ( P[2,1] * V_Next_GE(a_prime) + P[2,2] * V_Next_GU(a_prime) +
                P[2,3] * V_Next_BE(a_prime) + P[2,4] * V_Next_BU(a_prime))  )
            ubd_GU = min.( (1 + r_g)*agrid[a_idx], agrid[end] )
            res_GU = optimize(obj_GU, agrid[1], ubd_GU)       
            gg[a_idx,Kidx,2,1] = res_GU.minimizer
            Vnew[a_idx,Kidx,2,1] = (-1) * res_GU.minimum      


            ## Good && Employed
            obj_GE(a_prime) = -(  U( w_g*L_bar*(1-tau_g) + (1 + r_g) * agrid[a_idx] - a_prime + eps() ) + 
                β* ( P[1,1] * V_Next_GE(a_prime) + P[1,2] * V_Next_GU(a_prime) +
                P[1,3] * V_Next_BE(a_prime) + P[1,4] * V_Next_BU(a_prime))   )
            ubd_GE = min.( w_g * L_bar + (1 + r_g)*agrid[a_idx], agrid[end] )
            res_GE = optimize(obj_GE, agrid[1], ubd_GE)       
            gg[a_idx,Kidx,2,2] = res_GE.minimizer
            Vnew[a_idx,Kidx,2,2] = (-1) * res_GE.minimum
            
        end
    end

    return Vnew, gg
end

TV_Schumaker (generic function with 1 method)

In [84]:
function TV_Howard( V::Array{Float64,4},
    gg::Array{Float64,4},
    Kgrid::Array{Float64,1},
    agrid::Array{Float64,1},
    zgrid::Array{Float64,1},
    P::Array{Float64,2},
    na::Int64,
    nK::Int64,
    ne::Int64,
    nz::Int64, 
    Coef::Array{Float64,2},
    U,Neville)

    Vnew = copy(V)
    ### Where to assign Interpolation Value
    V_GE = zeros(na)
    V_GU = zeros(na)
    V_BE = zeros(na)
    V_BU = zeros(na)
    ####################################

    for (Kidx, Kval) in enumerate(Kgrid)
            
        r_g = α*zg*Kval^(α-1)*(Lgrid[1])^(1-α) - δ
        w_g = (1-α) * zg * Kval^α * (Lgrid[1])^(-α)
            
        r_b = α*zb*Kval^(α-1)*(Lgrid[2])^(1-α) - δ
        w_b = (1-α) * zb * Kval^α * (Lgrid[2])^(-α)

        tau_g = mu * ug / Lgrid[1]
        tau_b = mu * ub / Lgrid[2]
            
        K_Prime_g = ( Coef[1,1] + Coef[1,2]*log(Kval) ) |> exp
        K_Prime_b = ( Coef[2,1] + Coef[2,2]*log(Kval) ) |> exp
        # 여기 파트에서 
              
        K_Prime_g = max(K_Prime_g, Kgrid[1])
        K_Prime_b = max(K_Prime_b, Kgrid[1])
        K_Prime_g = min(K_Prime_g, Kgrid[end])
        K_Prime_b = min(K_Prime_b, Kgrid[end])
        
        # Use Neville Interpolation

        V_BU = map( x ->  Neville(Kgrid, V[x,:,1,1], K_Prime_b , nK), 1:na)
        V_BE = map( x ->  Neville(Kgrid, V[x,:,1,2], K_Prime_b , nK), 1:na)
        V_GU = map( x ->  Neville(Kgrid, V[x,:,2,1], K_Prime_g , nK), 1:na)
        V_GE = map( x ->  Neville(Kgrid, V[x,:,2,2], K_Prime_g , nK), 1:na)

        # Use Cubic Spline

        V_Next_GE = Spline1D( agrid, V_GE ,k = 3) 
        V_Next_GU = Spline1D( agrid, V_GU ,k = 3) 
        V_Next_BE = Spline1D( agrid, V_BE ,k = 3) 
        V_Next_BU = Spline1D( agrid, V_BU ,k = 3)  
    
        # Objective Functions 
    
        # zidx = 1 이면 오늘 Good / = 2 이면 오늘 Bad
        
        ## Bad && Unemployed

        Vnew[:,Kidx,1,1] = map( x ->  (  U( mu * w_b +  (1 + r_b ) * agrid[x] - gg[x,Kidx,1,1] + eps() ) + 
            β * ( P[4,1] * V_Next_GE(gg[x,Kidx,1,1]) + P[4,2] * V_Next_GU(gg[x,Kidx,1,1]) +
            P[4,3] * V_Next_BE(gg[x,Kidx,1,1]) + P[4,4] * V_Next_BU(gg[x,Kidx,1,1]) ) ) , 1:na) 

        ## Bad && Employed

        Vnew[:,Kidx,1,2] = map( x ->  (  U( w_b * L_bar * (1-tau_b) + (1 + r_b) * agrid[x] - gg[x,Kidx,1,2] + eps() ) + 
            β * ( P[3,1] * V_Next_GE( gg[x,Kidx,1,2] ) + P[3,2] * V_Next_GU( gg[x,Kidx,1,2] ) +
            P[3,3] * V_Next_BE( gg[x,Kidx,1,2] ) + P[3,4] * V_Next_BU( gg[x,Kidx,1,2] ) ) ) , 1:na  ) 


        ## Good && Unemployed    
        Vnew[:,Kidx,2,1] = map( x -> (  U( mu * w_g + (1 + r_g) * agrid[x] - gg[x,Kidx,2,1] + eps() ) + 
            β * ( P[2,1] * V_Next_GE(gg[x,Kidx,2,1]) + P[2,2] * V_Next_GU(gg[x,Kidx,2,1]) +
            P[2,3] * V_Next_BE(gg[x,Kidx,2,1]) + P[2,4] * V_Next_BU(gg[x,Kidx,2,1])) )  , 1:na )    


        ## Good && Employed
            
        Vnew[:,Kidx,2,2] = map( x ->  (  U( w_g*L_bar * (1-tau_g) + (1 + r_g) * agrid[x] - gg[x,Kidx,2,2] + eps() ) + 
            β * ( P[1,1] * V_Next_GE(gg[x,Kidx,2,2]) + P[1,2] * V_Next_GU(gg[x,Kidx,2,2]) +
            P[1,3] * V_Next_BE( gg[x,Kidx,2,2] ) + P[1,4] * V_Next_BU(gg[x,Kidx,2,2])) )  , 1:na )
          
    end

    return Vnew
end

TV_Howard (generic function with 1 method)

In [85]:
function TV_Howard_Schumaker( V::Array{Float64,4},
    gg::Array{Float64,4},
    Kgrid::Array{Float64,1},
    agrid::Array{Float64,1},
    zgrid::Array{Float64,1},
    P::Array{Float64,2},
    na::Int64,
    nK::Int64,
    ne::Int64,
    nz::Int64, 
    Coef::Array{Float64,2},
    U,Neville)

    Vnew = copy(V)
    ### Where to assign Interpolation Value
    V_GE = zeros(na)
    V_GU = zeros(na)
    V_BE = zeros(na)
    V_BU = zeros(na)
    ####################################

    for (Kidx, Kval) in enumerate(Kgrid)
            
        r_g = α*zg*Kval^(α-1)*(Lgrid[1])^(1-α) - δ
        w_g = (1-α) * zg * Kval^α * (Lgrid[1])^(-α)
            
        r_b = α*zb*Kval^(α-1)*(Lgrid[2])^(1-α) - δ
        w_b = (1-α) * zb * Kval^α * (Lgrid[2])^(-α)

        tau_g = mu * ug / Lgrid[1]
        tau_b = mu * ub / Lgrid[2]
            
        K_Prime_g = ( Coef[1,1] + Coef[1,2]*log(Kval) ) |> exp
        K_Prime_b = ( Coef[2,1] + Coef[2,2]*log(Kval) ) |> exp
        # 여기 파트에서 
              
        K_Prime_g = max(K_Prime_g, Kgrid[1])
        K_Prime_b = max(K_Prime_b, Kgrid[1])
        K_Prime_g = min(K_Prime_g, Kgrid[end])
        K_Prime_b = min(K_Prime_b, Kgrid[end])
        
        # Use Neville Interpolation

        V_BU = map( x ->  Neville(Kgrid, V[x,:,1,1], K_Prime_b , nK), 1:na)
        V_BE = map( x ->  Neville(Kgrid, V[x,:,1,2], K_Prime_b , nK), 1:na)
        V_GU = map( x ->  Neville(Kgrid, V[x,:,2,1], K_Prime_g , nK), 1:na)
        V_GE = map( x ->  Neville(Kgrid, V[x,:,2,2], K_Prime_g , nK), 1:na)

        # Use Cubic Spline

        V_Next_GE = SchumakerSpline.Schumaker( agrid, V_GE ) 
        V_Next_GU = SchumakerSpline.Schumaker( agrid, V_GU ) 
        V_Next_BE = SchumakerSpline.Schumaker( agrid, V_BE ) 
        V_Next_BU = SchumakerSpline.Schumaker( agrid, V_BU )  
    
        # Objective Functions 
    
   
        Vnew[:,Kidx,1,1] = map( x ->  (  U( mu * w_b +  (1 + r_b ) * agrid[x] - gg[x,Kidx,1,1] + eps() ) + 
            β * ( P[4,1] * V_Next_GE(gg[x,Kidx,1,1]) + P[4,2] * V_Next_GU(gg[x,Kidx,1,1]) +
            P[4,3] * V_Next_BE(gg[x,Kidx,1,1]) + P[4,4] * V_Next_BU(gg[x,Kidx,1,1]) ) ) , 1:na) 

        ## Bad && Employed

        Vnew[:,Kidx,1,2] = map( x ->  (  U( w_b * L_bar * (1-tau_b) + (1 + r_b) * agrid[x] - gg[x,Kidx,1,2] + eps() ) + 
            β * ( P[3,1] * V_Next_GE( gg[x,Kidx,1,2] ) + P[3,2] * V_Next_GU( gg[x,Kidx,1,2] ) +
            P[3,3] * V_Next_BE( gg[x,Kidx,1,2] ) + P[3,4] * V_Next_BU( gg[x,Kidx,1,2] ) ) ) , 1:na  ) 


        ## Good && Unemployed    
        Vnew[:,Kidx,2,1] = map( x -> (  U( mu * w_g + (1 + r_g) * agrid[x] - gg[x,Kidx,2,1] + eps() ) + 
            β * ( P[2,1] * V_Next_GE(gg[x,Kidx,2,1]) + P[2,2] * V_Next_GU(gg[x,Kidx,2,1]) +
            P[2,3] * V_Next_BE(gg[x,Kidx,2,1]) + P[2,4] * V_Next_BU(gg[x,Kidx,2,1])) )  , 1:na )    


        ## Good && Employed
            
        Vnew[:,Kidx,2,2] = map( x ->  (  U( w_g*L_bar * (1-tau_g) + (1 + r_g) * agrid[x] - gg[x,Kidx,2,2] + eps() ) + 
            β * ( P[1,1] * V_Next_GE(gg[x,Kidx,2,2]) + P[1,2] * V_Next_GU(gg[x,Kidx,2,2]) +
            P[1,3] * V_Next_BE( gg[x,Kidx,2,2] ) + P[1,4] * V_Next_BU(gg[x,Kidx,2,2])) )  , 1:na )  
          
    end

    return Vnew
end

TV_Howard_Schumaker (generic function with 1 method)

In [86]:
function VFI( Kgrid::Array{Float64,1},
    zgrid::Array{Float64,1},
    agrid::Array{Float64,1},
    P::Array{Float64,2},
    na::Int64,
    nK::Int64,
    ne::Int64,
    nz::Int64,
    U,Neville,
    Coef::Array{Float64,2},
    TV, TV_Howard ; Howard::Int64 = 1) 
    V_old = zeros(na,nK,nz,ne) # 순서는 철저히 논문의 순서를 따름
    GG_old = zeros(na,nK,nz,ne)
    GG_new = zeros(na,nK,nz,ne)
    err_V = one(Float64)
    err_G = one(Float64)
    iter = zero(Int64)
    V_new = copy(V_old)
    while  err_V > 1e-8 
        if iter % Howard == 0  
            V_new, GG_new = TV( V_old,Kgrid,agrid,zgrid,P,na,nK,ne,nz,Coef,U,Neville)
            err_G = (GG_new .- GG_old) .|> abs |> maximum
            copy!(GG_old,GG_new) 
        else 
            V_new = TV_Howard( V_old,GG_old,Kgrid,agrid,zgrid,P,na,nK,ne,nz,Coef,U,Neville)
        end
        err_V = (V_new .- V_old) .|> abs |> maximum
        copy!(V_old,V_new)
        iter += 1

        #if iter >=  Howard * 30 && err_G < 1e-5 
        #    break
        # end

        if iter > 3_000
            println("Something is Weird")
            break
        end
    end

    
return V_new , GG_new, iter
end

VFI (generic function with 1 method)

In [87]:
function VFI_Schumaker( Kgrid::Array{Float64,1},
    zgrid::Array{Float64,1},
    agrid::Array{Float64,1},
    P::Array{Float64,2},
    na::Int64,
    nK::Int64,
    ne::Int64,
    nz::Int64,
    U,Neville,
    Coef::Array{Float64,2},
    TV_Schumaker, TV_Howard_Schumaker ; Howard::Int64 = 1) 
    V_old = zeros(na,nK,nz,ne) # 순서는 철저히 논문의 순서를 따름
    GG_old = zeros(na,nK,nz,ne)
    GG_new = zeros(na,nK,nz,ne)
    err_V = one(Float64)
    err_G = one(Float64)
    iter = zero(Int64)
    V_new = copy(V_old)
    while  err_V > 1e-8  
        if iter % Howard == 0  
            V_new, GG_new = TV_Schumaker( V_old,Kgrid,agrid,zgrid,P,na,nK,ne,nz,Coef,U,Neville)
            err_G = (GG_new .- GG_old) .|> abs |> maximum
            copy!(GG_old,GG_new) 
        else 
            V_new = TV_Howard_Schumaker( V_old,GG_old,Kgrid,agrid,zgrid,P,na,nK,ne,nz,Coef,U,Neville)
        end
        err_V = (V_new .- V_old) .|> abs |> maximum
        copy!(V_old,V_new)
        iter += 1

        if iter >=  Howard * 30 && err_G < 1e-5 
            break
        end
        if iter > 3_000
            println("Something is Weird")
            break
        end
    end

    
return V_new , GG_new, iter
end

VFI_Schumaker (generic function with 1 method)

In [88]:
function VFI_Test( Kgrid::Array{Float64,1},
    zgrid::Array{Float64,1},
    agrid::Array{Float64,1},
    P::Array{Float64,2},
    na::Int64,
    nK::Int64,
    ne::Int64,
    nz::Int64,
    U,Neville,
    Coef::Array{Float64,2},
    TV, TV_Howard ; Howard::Int64 = 1) 
    V_old = zeros(na,ne,nK,nz) # 순서는 철저히 논문의 순서를 따름
    GG_old = zeros(na,ne,nK,nz)
    GG_new = zeros(na,ne,nK,nz)
    err_V = one(Float64)
    err_G = one(Float64)
    V_new = copy(V_old)
    for iter in 0:3000

        if iter > 10
            if iter % Howard == 0  
                V_new, GG_new = TV( V_old,Kgrid,agrid,zgrid,P,na,nK,ne,nz,Coef,U,Neville)
                err_G = (GG_new .- GG_old) .|> abs |> maximum
                err_V = (V_new .- V_old) .|> abs |> maximum
                copy!(GG_old,GG_new) 
                copy!(V_old,V_new)
                else 
                    V_new = TV_Howard( V_new,GG_new,Kgrid,agrid,zgrid,P,na,nK,ne,nz,Coef,U,Neville)
                    err_V = (V_new .- V_old) .|> abs |> maximum
                copy!(V_old,V_new)
                
            end
            
        else 
            V_new, GG_new = TV( V_old,Kgrid,agrid,zgrid,P,na,nK,ne,nz,Coef,U,Neville)
            err_V = (V_new .- V_old) .|> abs |> maximum
            err_G = (GG_new .- GG_old) .|> abs |> maximum
            copy!(GG_old,GG_new)
            copy!(V_old,V_new)
            
        end
        if iter % 100 == 0 
            println("Iteration is $iter")
            println("Value Function Error is $err_V")
            println("Policy Function Error is $err_G")
        end
        if err_V < 1e-6
            break
        end
    end

    
return V_new , GG_new
end

VFI_Test (generic function with 1 method)

In [89]:
function VFI_Test_Schumaker( Kgrid::Array{Float64,1},
    zgrid::Array{Float64,1},
    agrid::Array{Float64,1},
    P::Array{Float64,2},
    na::Int64,
    nK::Int64,
    ne::Int64,
    nz::Int64,
    U,Neville,
    Coef::Array{Float64,2},
    TV_Schumaker, 
    TV_Howard_Schumaker ; Howard::Int64 = 1) 
    V_old = zeros(na,ne,nK,nz) # 순서는 철저히 논문의 순서를 따름
    GG_old = zeros(na,ne,nK,nz)
    GG_new = zeros(na,ne,nK,nz)
    err_V = one(Float64)
    err_G = one(Float64)
    V_new = copy(V_old)
    for iter in 0:3000

        if iter > 10
            if iter % Howard == 0  
                V_new, GG_new = TV_Schumaker( V_old,Kgrid,agrid,zgrid,P,na,nK,ne,nz,Coef,U,Neville)
                err_G = (GG_new .- GG_old) .|> abs |> maximum
                err_V = (V_new .- V_old) .|> abs |> maximum
                copy!(GG_old,GG_new) 
                copy!(V_old,V_new)
                else 
                    V_new = TV_Howard_Schumaker( V_new,GG_new,Kgrid,agrid,zgrid,P,na,nK,ne,nz,Coef,U,Neville)
                    err_V = (V_new .- V_old) .|> abs |> maximum
                copy!(V_old,V_new)
                
            end
            
        else 
            V_new, GG_new = TV_Schumaker( V_old,Kgrid,agrid,zgrid,P,na,nK,ne,nz,Coef,U,Neville)
            err_V = (V_new .- V_old) .|> abs |> maximum
            err_G = (GG_new .- GG_old) .|> abs |> maximum
            copy!(GG_old,GG_new)
            copy!(V_old,V_new)
            
        end
        if iter % 100 == 0 
            println("Iteration is $iter")
            println("Value Function Error is $err_V")
            println("Policy Function Error is $err_G")
        end
        if err_V < 1e-6
            break
        end
    end

    
return V_new , GG_new
end

VFI_Test_Schumaker (generic function with 1 method)

In [90]:
function OLS(Y,X ;add_constant = 0 , print = 0 )
    # Input Y : Dependent Variable
    # Input X : Covariates
    # Input add_constant : 1 if we add one vector 
    # Output Estimator      : OLS Estimator 
    # Output SE             : Standard Error 
    # Output SE_robust      : Robust Standard Error 
    # Output sigma2         : Standard Error of Regression
    # Output fitted         : Fitted Value by model 

    Y = reshape(Y,length(Y),1);
    # Reshape data if X is one dimensional vector 
    if ndims(X) == 1
        X = reshape(X,length(X),1);
    end
    
nobs , nvar = size(X);
    
    if add_constant == 1
        X = [ones(nobs,1) X];
        nvar += 1;
    end

df = nobs - nvar;                # Degree of Freedom
XX = X'*X;
estimator = (XX)\(X'*Y);
fitted = X * estimator;
resid = Y - fitted;
SSR = sum(resid.^2);
sigma2 = SSR / (nobs-nvar);
inv_XX = inv(XX);
cov_mat = sigma2*inv_XX ;
SE = sqrt.(diag(cov_mat));

t_stat = estimator ./ SE;

Y_demean = Y .- mean(Y);
R2 = 1-SSR/sum(Y_demean.^2);
log_like = -nobs/2 * (1 + log(2*pi) + log(SSR/nobs) );
DW = sum(diff(resid, dims = 1).^2) ./ SSR; # Durbin Watson 
    if print == 1
    println(" ")
    println("Number of observations   " , nobs  )
    println("Number of regressors     " ,nvar)
    println("result")
        
    println("R-square     :    ", R2)
    println("Variance of s2:    ",sigma2)
    println("Log likelihood:    ",log_like)
    println("D-W statistics:    ",DW)
    end 
    
    return estimator
end

OLS (generic function with 1 method)

In [91]:
update_weight = 0.1
ERR = one(Float64)
iter = 0

0

In [92]:
A_Ind = zeros(NH,T)
A_Ind[:,1] .= kss;
Kt = zeros(T);
Lseq = zeros(T)
for t = 1:T 
    if zseq[t] == 1
        Lseq[t] = Lgrid[1]
    else
        Lseq[t] = Lgrid[2]
    end
end

In [94]:
Yt = zeros(T);

In [95]:
Kt[1] = mean(A_Ind[:,1])
Yt[1] = Zt[1] * Kt[1]^α * Lseq[1]^(1-α)

3.6670182234744244

In [96]:
function bilin_A( a::Float64, K::Float64, e0::Int64, z0::Int64 ,gg)
    
    if e0 == 1 # Today Employed
        if z0 == 1 # Today Good State
            pol = gg[:,:,2,2]
        else # Today is bad State
            pol = gg[:,:,1,2]
        end
            
    else # e0 == 0
        if z0 == 1
            pol = gg[:,:,2,1]
        else
            pol = gg[:,:,1,1]
        end 
    end

    a = max(a, amin)
    a = min(a, amax)    
    
    K = max(K,Kmin)
    K = min(K,Kmax)

    m2 = searchsortedlast(agrid, a)
    n2 = searchsortedlast(Kgrid, K)
    
    phi = zeros(4);
    f = zeros(4);
    d = zeros(4);
    # bilinear interpolation */
        
    if m2 < na && n2 < nK;
        d= [ agrid[m2+1] , agrid[m2] , Kgrid[n2+1], Kgrid[n2] ]       
        z1=( a - d[2])/(d[1]-d[2]);
        z2=( K -d[4])/(d[3]-d[4]);
        phi[1]=(1-z1)*(1-z2);
        phi[2]=z1*(1-z2);
        phi[3]=z1*z2;
        phi[4]=(1-z1)*z2;
        f[1]=pol[m2,n2];
        f[2]=pol[m2,n2+1];
        f[3]=pol[m2+1,n2+1];
        f[4]=pol[m2+1,n2];
        return dot(phi,f) ;
    elseif m2 == na && n2 == nK
        return (pol[m2,n2]);
    elseif m2 < na && n2 == nK  #linear interpolation */
        m1 = ( -agrid[m2] + a)/(agrid[m2+1] - agrid[m2])
        return ((1-m1)*pol[m2,nK]+m1*pol[m2+1,nK]);
    elseif n2 < nK && m2 == na;  #linear interpolation */
        n1 = (-Kgrid[n2] + K)/(Kgrid[n2+1] - Kgrid[n2]);
        return ((1-n1)*pol[na,n2]+n1*pol[na,n2+1]);
    end
end

bilin_A (generic function with 1 method)

In [97]:
XX = zeros(T - T0,3);
GG = zeros(na,nK,nz,ne);
VV = zeros(na,nK,nz,ne);

In [100]:
Coef =    [ 0.0 1.0 ; 0.0 1.0]

2×2 Matrix{Float64}:
 0.0  1.0
 0.0  1.0

In [ ]:
updata_weight = 0.1

0.1

In [ ]:
for iter = 1:1000
    println("VFI Start")
    ~ ,GG , ~ = VFI(  Kgrid, zgrid,agrid, P, na, nK, ne, nz, U, Neville, Coef, TV , TV_Howard ; Howard  = 7)  
    println("VFI End")

    if ERR < 1e-4
        A_Ind[:,1] = A_Ind[:,end]
        Kt[1] = mean(A_Ind[:,1])
    end
    itp =  LinearInterpolation( ( agrid, Kgrid, [0,1],[0,1]), GG, extrapolation_bc = Line() )
        for t = 2:T
            A_Ind[:,t] = map( x -> itp(A_Ind[x,t-1], Kt[t-1],  zseq[t] , Ind_shock[x,t] ) , 1:NH )
            Kt[t] = mean(A_Ind[:,t])
            Kt[t] = max(Kgrid[1],Kt[t])
            Kt[t] = min(Kgrid[end],Kt[t])
        end 
     
    XX[:,1] = Kt[T0:T-1]
    XX[:,2] = Kt[T0+1:T]
    XX[:,3] = Zt[T0:T-1]
    XX_Good = XX[findall(x->x == 1.01, XX[:,3]),:] 
    XX_Bad = XX[findall(x->x == 0.99, XX[:,3]),:] 
    EST_G = OLS(log.(XX_Good[:,2]), log.(XX_Good[:,1]) ;add_constant = 1 , print = 0 ) 
    EST_B = OLS(log.(XX_Bad[:,2]), log.(XX_Bad[:,1]) ;add_constant = 1 , print = 0 ) 
    New_Coef = [EST_G' ; EST_B']
    println(New_Coef)
    println(Coef)
    
    ERR = (Coef - New_Coef) .|> abs |> maximum 
    println("Iteration is $iter", "   ", "Error level is ", " ", "$ERR")  
    
    Coef = (1-update_weight) * Coef + update_weight * New_Coef
    iter += 1
    if ERR < 1e-8
        break
    end
end

VFI Start
VFI End
[0.510594470447961 0.8616340459525939; 0.4787631494359709 0.8688753393076438]
[0.0 1.0; 0.0 1.0]
Iteration is 1   Error level is  0.510594470447961
VFI Start
VFI End
[0.2634563729473977 0.92896881016798; 0.24668422547207453 0.9321106119914312]
[0.051059447044796105 0.9861634045952594; 0.04787631494359709 0.9868875339307644]
Iteration is 2   Error level is  0.21239692590260162
VFI Start
VFI End
[0.22000687909961983 0.9407899711994148; 0.20558361095304498 0.9433012613381256]
[0.07229913963505627 0.9804439451525315; 0.06775710599644484 0.9814098417368311]
Iteration is 3   Error level is  0.14770773946456356
VFI Start
VFI End
[0.19791179846706036 0.9468021780234172; 0.18469169613700626 0.9489931095123967]
[0.08706991358151263 0.9764785477572199; 0.08153975649210485 0.9775989836969605]
Iteration is 4   Error level is  0.11084188488554773
VFI Start
VFI End
[0.1843437929650839 0.9504938895514979; 0.17181647006643813 0.9525022499935478]
[0.09815410207006742 0.9735109107838396

In [60]:
for t = 1:T
    if zseq[t] == 1
        KKt[t] = New_Coef[1,1] + New_Coef[1,2] * log(Kt[t]) |> exp
            else
        KKt[t] = New_Coef[2,1] + New_Coef[2,2] * log(Kt[t]) |> exp
    end
end

In [ ]:
Coef[1,1] |> println 
Coef[1,2] |> println
Coef[2,1] |> println
Coef[2,2] |> println

0.13860336537999843
0.9629356729060098
0.11883689639824496
0.9670082535155495


In [57]:
XX[:,1] = Kt[T0:T-1]
    XX[:,2] = Kt[T0+1:T]
    XX[:,3] = Zt[T0:T-1]
    XX_Good = XX[findall(x->x == 1.01, XX[:,3]),:] 
    XX_Bad = XX[findall(x->x == 0.99, XX[:,3]),:] 
    EST_G = OLS(log.(XX_Good[:,2]), log.(XX_Good[:,1]) ;add_constant = 1 , print = 1 ) 
    EST_B = OLS(log.(XX_Bad[:,2]), log.(XX_Bad[:,1]) ;add_constant = 1 , print = 1 ) 

 
Number of observations   517
Number of regressors     2
result
R-square     :    0.9953995261659454
Variance of s2:    2.643307058838391e-6
Log likelihood:    2587.450233191279
D-W statistics:    2.0062784989471947
 
Number of observations   483
Number of regressors     2
result
R-square     :    0.9906497882397615
Variance of s2:    4.263283692195773e-6
Log likelihood:    2301.91598086442
D-W statistics:    1.8079831310734924


2×1 Matrix{Float64}:
 0.1260768752912457
 0.9650481216227436

In [39]:
Coef = [0.0 1.0 ; 0.0 1.0 ]

2×2 Matrix{Float64}:
 0.0  1.0
 0.0  1.0

In [56]:
for iter = 1:1000
    println("VFI Start")
    ~ ,GG , ~ = VFI(  Kgrid, zgrid,agrid, P, na, nK, ne, nz, U, Neville, Coef, TV , TV_Howard ; Howard  = 6)  
    println("VFI End")

    if ERR < 1e-4
        A_Ind[:,1] = A_Ind[:,end]
        Kt[1] = mean(A_Ind[:,1])
    end
    itp =  LinearInterpolation( ( agrid, Kgrid, [0,1],[0,1]), GG, extrapolation_bc = Line() )
    for t = 2:T
        A_Ind[:,t] = map( x -> itp(A_Ind[x,t-1], Kt[t-1],  zseq[t] , Ind_shock[x,t] ) , 1:NH )
        Kt[t] = mean(A_Ind[:,t])
        Kt[t] = max(Kgrid[1],Kt[t])
        Kt[t] = min(Kgrid[end],Kt[t])
    end 
     
    XX[:,1] = Kt[T0:T-1]
    XX[:,2] = Kt[T0+1:T]
    XX[:,3] = Zt[T0:T-1]
    XX_Good = XX[findall(x->x == 1.01, XX[:,3]),:] 
    XX_Bad = XX[findall(x->x == 0.99, XX[:,3]),:] 
    EST_G = OLS(log.(XX_Good[:,2]), log.(XX_Good[:,1]) ;add_constant = 1 , print = 0 ) 
    EST_B = OLS(log.(XX_Bad[:,2]), log.(XX_Bad[:,1]) ;add_constant = 1 , print = 0 ) 
    New_Coef = [EST_G' ; EST_B']
    println(New_Coef)
    println(Coef)
    
    ERR_New = (Coef - New_Coef) .|> abs |> maximum 
    
    println("Iteration is $iter", "   ", "Error level is ", " ", "$ERR_New")  
    iter += 1
    Coef = (1-update_weight) * Coef + update_weight * New_Coef
    ERR = ERR_New
    if ERR < 1e-8
        break
    end
end

VFI Start
VFI End
[0.14536995443734427 0.9611117571140483; 0.12585864447648928 0.9651056924107085]
[0.14528455314962183 0.961134487268861; 0.1257324483502089 0.9651392172521406]
Iteration is 1   Error level is  0.0001261961262803979
VFI Start
VFI End
[0.14536625960703556 0.9611126062526707; 0.12585848223897037 0.9651055880451833]
[0.14529309327839407 0.9611322142533797; 0.12574506796283694 0.9651358647679974]
Iteration is 2   Error level is  0.0001134142761334278
VFI Start
VFI End
[0.14536235027543348 0.9611135660312811; 0.1258567812776947 0.965105949676779]
[0.1453004099112582 0.9611302534533088; 0.1257564093904503 0.965132837095716]
Iteration is 3   Error level is  0.00010037188724440482
VFI Start
VFI End
[0.145358470658955 0.961114552316268; 0.12585425723991023 0.9651065678857921]
[0.14530660394767572 0.9611285847111061; 0.12576644657917474 0.9651301483538223]
Iteration is 4   Error level is  8.781066073548782e-5
VFI Start
VFI End
[0.14540549860621513 0.9611030156386603; 0.125895707

In [35]:
New_Coef = copy(Coef)

2×2 Matrix{Float64}:
 0.0  1.0
 0.0  1.0

In [63]:
using Plots

In [69]:
plot(2:T, Kt[2:T], label = "Actual Aggregate Capital", 
    size = (1600,900), xlabel = "Time Period", ylabel = "Capital")
plot!(2:T,KKt[1:T-1] , label = "Predicted Aggregate Captital")
title!("Replication Figure of Maliar et al(2010)")
savefig("/content/drive/MyDrive/Figure_1.png")

In [68]:
GG

100×6×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
    0.0            0.0          …    0.0            0.0
    4.4381e-14     1.4309e-13        5.40786e-14    3.3351e-14
    5.8374e-14     3.51545e-14       7.36723e-14    5.61604e-14
    5.45149e-14    5.41603e-14       5.31423e-14    4.43557e-14
    8.23443e-15    8.42635e-14       8.31247e-14    5.84472e-14
    2.56193e-14    5.90779e-14  …    2.5184e-14     1.72245e-13
    1.33933e-14    4.97847e-14       1.46009e-13    8.75369e-14
    3.94016e-14    6.28468e-14       7.70109e-14    1.01185e-13
    3.25148e-14    3.80757e-14       9.44498e-14    1.98441e-14
    3.33868e-14    4.21639e-14       9.05577e-14    3.27493e-14
    1.51924e-14    8.1636e-15   …    3.22072e-14    6.70901e-14
    8.40694e-14    4.71078e-15       6.87207e-14    3.48932e-14
    3.10669e-14    9.12116e-14       2.21699e-14    5.71106e-14
    ⋮                           ⋱                   ⋮
  441.369        438.22            432.974        431.886
  477.827        474.

In [70]:
using JLD2

In [72]:
zseq

1100-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 1
 0
 1
 1
 1
 1
 1
 ⋮
 1
 1
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1

In [74]:
jldsave("/content/drive/MyDrive/Krusell_Smith_Result.jld2"; 
GG, A_Ind, Kt, KKt , Ind_shock, zseq, Zt, Coef) 